In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

import tensorflow as tf
import numpy as np
from tensorflow import keras

import larq as lq
import larq_compute_engine as lce

import matplotlib.pyplot as plt

from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from notebooks.helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstraping_partitions_full_int_q,
    )

2024-09-22 15:11:38.263223: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 15:11:38.311144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 15:11:38.311175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 15:11:38.312191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 15:11:38.320752: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-22 15:11:38.321830: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-09-22 15:11:48.709926: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22 15:11:48.710329: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-22 15:11:48.824096: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-09-22 15:11:48.833182: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

### HPO

In [5]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes, num_of_layers):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.num_of_conv_layers = num_of_layers

    def build(self, hp):
        model = tf.keras.Sequential()
        
        # Hyperparameters
        dense_units = hp.Int('1st_dense_units', min_value=4, max_value=32, step=4)
        dense_activation = hp.Choice('2nd_dense_activation', values=['softmax', 'sigmoid'])
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

        # Model architecture
        # Tune number of layers
        for i in range(self.num_of_conv_layers):
            if i == 0:
                model.add(lq.layers.QuantConv1D(
                    filters=hp.Int(f'filters_{i}', min_value=2, max_value=8, step=2), 
                    kernel_size=hp.Choice(f'kernel_size_{i}', values=[2, 3, 5]),
                    input_shape=self.input_shape,
                    kernel_quantizer="ste_sign",
                    kernel_constraint="weight_clip",
                    use_bias=False
                ))
            else:
                model.add(lq.layers.QuantConv1D(
                    filters=hp.Int(f'filters_{i}', min_value=2, max_value=8, step=2), 
                    kernel_size=hp.Choice(f'kernel_size_{i}', values=[2, 3, 5]),
                    input_quantizer="ste_sign",
                    kernel_quantizer="ste_sign",
                    kernel_constraint="weight_clip",
                    use_bias=False
                ))
            model.add(tf.keras.layers.MaxPooling1D(pool_size=hp.Int(f'pull_size_{i}', min_value=4, max_value=10, step=2)))
            model.add(tf.keras.layers.BatchNormalization(scale=False))

        model.add(tf.keras.layers.Flatten())

        model.add(lq.layers.QuantDense(
            units=dense_units,
            kernel_quantizer="ste_sign",
            input_quantizer="ste_sign",
            kernel_constraint="weight_clip",
            use_bias=False
        ))
        model.add(tf.keras.layers.BatchNormalization(scale=False))

        model.add(lq.layers.QuantDense(
            units=self.num_classes,
            kernel_quantizer="ste_sign",
            input_quantizer="ste_sign",
            kernel_constraint="weight_clip",
            use_bias=False
        ))
        model.add(tf.keras.layers.BatchNormalization(scale=False))
        model.add(tf.keras.layers.Activation(dense_activation))
       
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        return model


#### Tune 1 conv2d layer CNN

In [6]:
input_shape = (48000,1)
num_classes = 2

num_of_conv_layers = 1
tuner_1_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_conv_layers),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='../hpo_tuner/bnn',
    project_name='bnn_time_series_1_conv2d_tuning'
)
tuner_1_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_1_layer_cnn.results_summary()

Trial 5 Complete [00h 07m 54s]
val_accuracy: 0.781521737575531

Best val_accuracy So Far: 0.8369565010070801
Total elapsed time: 01h 25m 45s
Results summary
Results in ../hpo_tuner/bnn/bnn_time_series_1_conv2d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
1st_dense_units: 28
2nd_dense_activation: sigmoid
learning_rate: 0.0011554054814026313
filters_0: 8
kernel_size_0: 5
pull_size_0: 6
Score: 0.8369565010070801

Trial 3 summary
Hyperparameters:
1st_dense_units: 12
2nd_dense_activation: sigmoid
learning_rate: 0.004018615305666302
filters_0: 2
kernel_size_0: 5
pull_size_0: 10
Score: 0.8018116056919098

Trial 1 summary
Hyperparameters:
1st_dense_units: 28
2nd_dense_activation: softmax
learning_rate: 0.0023880453124178174
filters_0: 2
kernel_size_0: 2
pull_size_0: 8
Score: 0.7981884181499481

Trial 4 summary
Hyperparameters:
1st_dense_units: 32
2nd_dense_activation: softmax
learning_rate: 0.0016098333118373667
filters_0: 8
ke

#### Tune 2 conv2d layers CNN

In [7]:
num_of_layers = 2
tuner_2_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_layers),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='../hpo_tuner/bnn',
    project_name='bnn_time_series_2_conv2d_tuning'
)
tuner_2_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_2_layer_cnn.results_summary()

Trial 10 Complete [00h 08m 37s]
val_accuracy: 0.8355072438716888

Best val_accuracy So Far: 0.8416666686534882
Total elapsed time: 01h 23m 58s
Results summary
Results in ../hpo_tuner/bnn/bnn_time_series_2_conv2d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
1st_dense_units: 8
2nd_dense_activation: softmax
learning_rate: 0.0038803956106591635
filters_0: 8
kernel_size_0: 2
pull_size_0: 4
filters_1: 8
kernel_size_1: 3
pull_size_1: 6
Score: 0.8416666686534882

Trial 04 summary
Hyperparameters:
1st_dense_units: 28
2nd_dense_activation: softmax
learning_rate: 0.00032847506986532663
filters_0: 6
kernel_size_0: 5
pull_size_0: 10
filters_1: 8
kernel_size_1: 5
pull_size_1: 6
Score: 0.8365942239761353

Trial 09 summary
Hyperparameters:
1st_dense_units: 32
2nd_dense_activation: softmax
learning_rate: 0.0030906537787876503
filters_0: 2
kernel_size_0: 5
pull_size_0: 4
filters_1: 2
kernel_size_1: 5
pull_size_1: 8
Score: 0.835507243871

#### Tune 3 conv2d layers CNN

In [8]:
num_of_layers = 3
tuner_3_layer_cnn = RandomSearch(
    CNNHyperModel(input_shape, num_classes, num_of_layers),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='../hpo_tuner/bnn',
    project_name='bnn_time_series_3_conv2d_tuning'
)
tuner_3_layer_cnn.search(train_dataset, epochs=5, validation_data=val_dataset)
tuner_3_layer_cnn.results_summary()

Trial 10 Complete [00h 08m 03s]
val_accuracy: 0.8271739184856415

Best val_accuracy So Far: 0.8547101318836212
Total elapsed time: 01h 26m 28s
Results summary
Results in ../hpo_tuner/bnn/bnn_time_series_3_conv2d_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
1st_dense_units: 12
2nd_dense_activation: softmax
learning_rate: 0.004770118252590703
filters_0: 8
kernel_size_0: 5
pull_size_0: 4
filters_1: 6
kernel_size_1: 5
pull_size_1: 6
filters_2: 2
kernel_size_2: 3
pull_size_2: 6
Score: 0.8547101318836212

Trial 02 summary
Hyperparameters:
1st_dense_units: 32
2nd_dense_activation: sigmoid
learning_rate: 0.002045838206464179
filters_0: 2
kernel_size_0: 5
pull_size_0: 8
filters_1: 8
kernel_size_1: 5
pull_size_1: 8
filters_2: 6
kernel_size_2: 2
pull_size_2: 10
Score: 0.8416666686534882

Trial 04 summary
Hyperparameters:
1st_dense_units: 8
2nd_dense_activation: sigmoid
learning_rate: 0.0031380270528399827
filters_0: 4
kernel_size

In [12]:
tuner_1_layer_cnn.search_space_summary()


Search space summary
Default search space size: 6
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
kernel_size_0 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
pull_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}


In [13]:
tuner_2_layer_cnn.search_space_summary()

Search space summary
Default search space size: 9
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
kernel_size_0 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
pull_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
kernel_size_1 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
pull_size_1 (Int)
{'default'

In [14]:
tuner_3_layer_cnn.search_space_summary()

Search space summary
Default search space size: 12
1st_dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 32, 'step': 4, 'sampling': 'linear'}
2nd_dense_activation (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
kernel_size_0 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
pull_size_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 2, 'sampling': 'linear'}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 2, 'sampling': 'linear'}
kernel_size_1 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3, 5], 'ordered': True}
pull_size_1 (Int)
{'default

### Model 3

In [15]:
best_hps = tuner_3_layer_cnn.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

Best Hyperparameters:
1st_dense_units: 12
2nd_dense_activation: softmax
learning_rate: 0.004770118252590703
filters_0: 8
kernel_size_0: 5
pull_size_0: 4
filters_1: 6
kernel_size_1: 5
pull_size_1: 6
filters_2: 2
kernel_size_2: 3
pull_size_2: 6


In [18]:
model_3 = tf.keras.models.Sequential()

# The first layer, only the weights are quantized while activations are left full-precision
model_3.add(lq.layers.QuantConv1D(filters=8, 
                                  kernel_size=5,
                                  kernel_quantizer="ste_sign",
                                  kernel_constraint="weight_clip",
                                  use_bias=False,
                                  input_shape=(48000,1)))
model_3.add(tf.keras.layers.MaxPooling1D(4))
model_3.add(tf.keras.layers.BatchNormalization(scale=False))

model_3.add(lq.layers.QuantConv1D(
                    filters=6, 
                    kernel_size=5,
                    input_quantizer="ste_sign",
                    kernel_quantizer="ste_sign",
                    kernel_constraint="weight_clip",
                    use_bias=False
                ))
model_3.add(tf.keras.layers.MaxPooling1D(6))
model_3.add(tf.keras.layers.BatchNormalization(scale=False))

model_3.add(lq.layers.QuantConv1D(
                    filters=2, 
                    kernel_size=3,
                    input_quantizer="ste_sign",
                    kernel_quantizer="ste_sign",
                    kernel_constraint="weight_clip",
                    use_bias=False
                ))
model_3.add(tf.keras.layers.MaxPooling1D(6))
model_3.add(tf.keras.layers.BatchNormalization(scale=False))

model_3.add(tf.keras.layers.Flatten())

model_3.add(lq.layers.QuantDense(units=12, 
                                 use_bias=False, 
                                 input_quantizer="ste_sign",
                                 kernel_quantizer="ste_sign",
                                 kernel_constraint="weight_clip"))
model_3.add(tf.keras.layers.BatchNormalization(scale=False))

model_3.add(lq.layers.QuantDense(units=2, 
                                 use_bias=False, 
                                 input_quantizer="ste_sign",
                                 kernel_quantizer="ste_sign",
                                 kernel_constraint="weight_clip"))
model_3.add(tf.keras.layers.BatchNormalization(scale=False))
model_3.add(tf.keras.layers.Activation("softmax"))

model_3.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quant_conv1d_6 (QuantConv1  (None, 47996, 8)          40        
 D)                                                              
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 11999, 8)          0         
 g1D)                                                            
                                                                 
 batch_normalization_10 (Ba  (None, 11999, 8)          24        
 tchNormalization)                                               
                                                                 
 quant_conv1d_7 (QuantConv1  (None, 11995, 6)          240       
 D)                                                              
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 1999, 6)          

In [22]:
model_3_20_epochs = tf.keras.models.clone_model(model_3)
model_3_20_epochs.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004770118252590703),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# EPOCHS = 1
EPOCHS = 20
history = model_3_20_epochs.fit(
    train_dataset,
    epochs=EPOCHS
)

print("Validation dataset accuracy:")
val_loss, val_acc = model_3_20_epochs.evaluate(x_val_np, y_val_np)

print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_3_20_epochs, x_val_np, y_val_np, model_format="keras")

Epoch 1/20
353/353 [==============================] - 57s 156ms/step - loss: 0.4933 - accuracy: 0.7528
Epoch 2/20
353/353 [==============================] - 52s 146ms/step - loss: 0.4233 - accuracy: 0.7926
Epoch 3/20
353/353 [==============================] - 57s 159ms/step - loss: 0.4041 - accuracy: 0.8012
Epoch 4/20
353/353 [==============================] - 57s 159ms/step - loss: 0.3959 - accuracy: 0.8094
Epoch 5/20
353/353 [==============================] - 57s 159ms/step - loss: 0.3936 - accuracy: 0.8109
Epoch 6/20
353/353 [==============================] - 57s 159ms/step - loss: 0.3828 - accuracy: 0.8162
Epoch 7/20
353/353 [==============================] - 57s 158ms/step - loss: 0.3883 - accuracy: 0.8151
Epoch 8/20
353/353 [==============================] - 57s 160ms/step - loss: 0.4011 - accuracy: 0.8092
Epoch 9/20
353/353 [==============================] - 57s 158ms/step - loss: 0.3955 - accuracy: 0.8116
Epoch 10/20
353/353 [==============================] - 57s 160ms/step - l

In [23]:
model_3_10_epochs = tf.keras.models.clone_model(model_3)
model_3_10_epochs.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004770118252590703),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# EPOCHS = 1
EPOCHS = 10
history = model_3_10_epochs.fit(
    train_dataset,
    epochs=EPOCHS
)

print("Validation dataset accuracy:")
val_loss, val_acc = model_3_10_epochs.evaluate(x_val_np, y_val_np)

print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_3_10_epochs, x_val_np, y_val_np, model_format="keras")


Epoch 1/10


/home/polina/newname/.venv/lib/python3.9/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


353/353 [==============================] - 75s 204ms/step - loss: 0.5035 - accuracy: 0.7340
Epoch 2/10
353/353 [==============================] - 71s 198ms/step - loss: 0.4179 - accuracy: 0.8069
Epoch 3/10
353/353 [==============================] - 65s 181ms/step - loss: 0.4004 - accuracy: 0.8170
Epoch 4/10
353/353 [==============================] - 57s 160ms/step - loss: 0.4246 - accuracy: 0.7921
Epoch 5/10
353/353 [==============================] - 57s 160ms/step - loss: 0.4132 - accuracy: 0.7997
Epoch 6/10
353/353 [==============================] - 57s 160ms/step - loss: 0.3688 - accuracy: 0.8293
Epoch 7/10
353/353 [==============================] - 58s 162ms/step - loss: 0.3336 - accuracy: 0.8529
Epoch 8/10
353/353 [==============================] - 57s 161ms/step - loss: 0.3379 - accuracy: 0.8535
Epoch 9/10
353/353 [==============================] - 59s 164ms/step - loss: 0.3348 - accuracy: 0.8552
Epoch 10/10
353/353 [==============================] - 57s 159ms/step - loss: 0.3277

In [25]:
model_3_5_epochs = tf.keras.models.clone_model(model_3)
model_3_5_epochs.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004770118252590703),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# EPOCHS = 1
EPOCHS = 5
history = model_3_5_epochs.fit(
    train_dataset,
    epochs=EPOCHS
)

print("Validation dataset accuracy:")
val_loss, val_acc = model_3_5_epochs.evaluate(x_val_np, y_val_np)

print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(model_3_5_epochs, x_val_np, y_val_np, model_format="keras")


Epoch 1/5
353/353 [==============================] - 62s 161ms/step - loss: 0.4439 - accuracy: 0.7802
Epoch 2/5
353/353 [==============================] - 58s 161ms/step - loss: 0.3351 - accuracy: 0.8516
Epoch 3/5
353/353 [==============================] - 57s 159ms/step - loss: 0.3329 - accuracy: 0.8591
Epoch 4/5
353/353 [==============================] - 58s 161ms/step - loss: 0.3706 - accuracy: 0.8362
Epoch 5/5
353/353 [==============================] - 58s 161ms/step - loss: 0.3681 - accuracy: 0.8393
Validation dataset accuracy:
44/44 [==============================] - 2s 31ms/step - loss: 0.6038 - accuracy: 0.7906
Validation dataset:
44/44 [==============================] - 2s 31ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 79.06%
Recall: 39.41%
Precision: 98.41%
F1-score: 56.28%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.5561581623457317

Get 100 bootstrap samples from dataset wi

In [26]:
print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(model_3_20_epochs, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(model_3_20_epochs, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)



Test dataset:
44/44 [==============================] - 1s 32ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.780648207060149

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.7865660474085666
Time of one prediction for Test dataset:
Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%

Time to make a prediction for a single data point
Mean: 0.051 seconds
Max: 0.238 seconds
Min: 0.045 seconds


In [43]:
MODEL_FILE_NAME = "../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model.keras"
model_3_20_epochs.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "KB")

Model file name:  ../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model.keras
File size: 179.52 Kilobytes


#### Final model

Third model has good f1-score and was small (179 Kb)

I will keep this model as the final one

Show latent weights of model:

In [30]:
import sys

In [44]:
def display_weights(model):
    for layer in model.layers:
        if isinstance(layer, lq.layers.QuantConv1D) or isinstance(layer, lq.layers.QuantDense):
            weights = layer.get_weights()
            if weights:
                print(f"Layer: {layer.name}")
                for weight in weights:
                    print(f"Shape: {weight.shape}")
                    # Print the unique values to see if they are binarized
                    unique_values = np.unique(weight)
                    print(f"Unique values: {unique_values}\n")
                    # Print the first few weights for inspection
                    print(f"First few weights: {weight.flatten()[:10]}\n")

# Display the weights of the binarized CNN model


def model_memory_usage(model):
    total_size = 0
    for layer in model.layers:
        if isinstance(layer, (lq.layers.QuantConv1D, lq.layers.QuantDense)):
            weights = layer.get_weights()
            for weight in weights:
                size = sys.getsizeof(weight)
                total_size += size
                print(f"Layer: {layer.name}, Weight shape: {weight.shape}, Size: {size} bytes")

    print(f"Total memory usage: {total_size} bytes")

# Display the weights of the binarized CNN model
display_weights(model_3_20_epochs)

# Check the memory usage of the model
model_memory_usage(model_3_20_epochs)

Layer: quant_conv1d_6
Shape: (5, 1, 8)
Unique values: [-1.  1.]

First few weights: [-1. -1.  1. -1. -1. -1. -1.  1. -1. -1.]

Layer: quant_conv1d_7
Shape: (5, 8, 6)
Unique values: [-1.  1.]

First few weights: [-1. -1. -1. -1.  1.  1.  1.  1. -1. -1.]

Layer: quant_conv1d_8
Shape: (3, 6, 2)
Unique values: [-1.  1.]

First few weights: [-1. -1.  1.  1.  1.  1.  1.  1. -1. -1.]

Layer: quant_dense_4
Shape: (664, 12)
Unique values: [-1.  1.]

First few weights: [ 1. -1.  1. -1.  1.  1.  1. -1. -1. -1.]

Layer: quant_dense_5
Shape: (12, 2)
Unique values: [-1.  1.]

First few weights: [ 1. -1. -1.  1.  1. -1. -1.  1.  1.  1.]

Layer: quant_conv1d_6, Weight shape: (5, 1, 8), Size: 304 bytes
Layer: quant_conv1d_7, Weight shape: (5, 8, 6), Size: 1104 bytes
Layer: quant_conv1d_8, Weight shape: (3, 6, 2), Size: 288 bytes
Layer: quant_dense_4, Weight shape: (664, 12), Size: 32000 bytes
Layer: quant_dense_5, Weight shape: (12, 2), Size: 224 bytes
Total memory usage: 33920 bytes


Show binary weights of model:

In [45]:
with lq.context.quantized_scope(True):
    model_3_20_epochs.save("../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras")  # save binarized weights h5
    weights = model_3_20_epochs.get_weights()  # get binarized weights

In [6]:
binarized_weights_model = tf.keras.models.load_model("../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras")

In [47]:
# Display the weights of the binarized CNN model
display_weights(binarized_weights_model)

# Check the memory usage of the model
model_memory_usage(binarized_weights_model)

Layer: quant_conv1d_6
Shape: (5, 1, 8)
Unique values: [-1.  1.]

First few weights: [-1. -1.  1. -1. -1. -1. -1.  1. -1. -1.]

Layer: quant_conv1d_7
Shape: (5, 8, 6)
Unique values: [-1.  1.]

First few weights: [-1. -1. -1. -1.  1.  1.  1.  1. -1. -1.]

Layer: quant_conv1d_8
Shape: (3, 6, 2)
Unique values: [-1.  1.]

First few weights: [-1. -1.  1.  1.  1.  1.  1.  1. -1. -1.]

Layer: quant_dense_4
Shape: (664, 12)
Unique values: [-1.  1.]

First few weights: [ 1. -1.  1. -1.  1.  1.  1. -1. -1. -1.]

Layer: quant_dense_5
Shape: (12, 2)
Unique values: [-1.  1.]

First few weights: [ 1. -1. -1.  1.  1. -1. -1.  1.  1.  1.]

Layer: quant_conv1d_6, Weight shape: (5, 1, 8), Size: 304 bytes
Layer: quant_conv1d_7, Weight shape: (5, 8, 6), Size: 1104 bytes
Layer: quant_conv1d_8, Weight shape: (3, 6, 2), Size: 288 bytes
Layer: quant_dense_4, Weight shape: (664, 12), Size: 32000 bytes
Layer: quant_dense_5, Weight shape: (12, 2), Size: 224 bytes
Total memory usage: 33920 bytes


In [48]:
print("Model file name: ", "../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras")
convert_bytes(get_file_size("../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras"), "MB")

Model file name:  ../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras
File size: 0.175 Megabytes


### larq TFLite

In [49]:
lce_model = lce.convert_keras_model(model_3_20_epochs)

INFO:tensorflow:Assets written to: /tmp/tmptv5xzbi5/assets


INFO:tensorflow:Assets written to: /tmp/tmptv5xzbi5/assets
2024-07-27 11:28:25.962286: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmptv5xzbi5
2024-07-27 11:28:25.974119: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-07-27 11:28:25.974149: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmptv5xzbi5
2024-07-27 11:28:25.977605: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-27 11:28:25.994514: I external/org_tensorflow/tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-07-27 11:28:2

Check time of prdiction by bnn tflite model

In [50]:
import time

In [51]:
exec_time = []
y_pred_all = []
for i in range(1, len(x_test_np)):
    # print(i)
    start_time = time.time()
    interpreter = lce.testing.Interpreter(lce_model)
    y_pred_prob = interpreter.predict(x_test_np[i-1:i], verbose=0)
    y_pred = tf.argmax(y_pred_prob, axis=1).numpy()
    y_pred_all.extend(y_pred)
    stop_time = time.time() - start_time
    # print(stop_time)
    exec_time.append(stop_time)
print("mean_time of one prediction, s: ", sum(exec_time) / len(exec_time))

mean_time of one prediction, s:  0.004089654690917881


In [52]:
y_test_np

array([0, 0, 0, ..., 1, 0, 1], dtype=int32)

In [53]:
evaluate_prediction(y_pred_all, y_test_np[:-1])

Accuracy: 84.34%
Recall: 72.42%
Precision: 84.46%
F1-score: 77.98%


write model in file

In [54]:
TF_LITE_LARQ_MODEL_FILE_NAME = "../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_lq.tflite"
open(TF_LITE_LARQ_MODEL_FILE_NAME, "wb").write(lce_model)

42144

In [62]:
TF_LITE_LARQ_MODEL_FILE_NAME = "../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_lq.tflite"
print("Model file name: ", TF_LITE_LARQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_LARQ_MODEL_FILE_NAME), "KB")

Model file name:  ../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_lq.tflite
File size: 41.156 Kilobytes


read model from file

In [97]:
interpreter = tf.lite.Interpreter(model_path=TF_LITE_LARQ_MODEL_FILE_NAME)

I dont know how to predict with the loaded model

### tf TFLite

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(binarized_weights_model)
tflite_model = converter.convert()

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_tf_lite.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

INFO:tensorflow:Assets written to: /tmp/tmprvzpldqr/assets


INFO:tensorflow:Assets written to: /tmp/tmprvzpldqr/assets
2024-09-22 15:14:04.714870: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 15:14:04.714957: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 15:14:04.715235: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprvzpldqr
2024-09-22 15:14:04.722829: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 15:14:04.722931: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmprvzpldqr
2024-09-22 15:14:04.740618: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 15:14:04.833087: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmprvzpldqr
2024-09-22 15:14:04.862306: I tensorflow/cc/saved_model/loader.cc:316] SavedModel


Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.780648207060149

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.7671719061395149

Time for Test dataset:
Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.009 seconds
Min: 0.003 seconds


Model file name:  ../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_tf_lite.tflite
File size: 44.512 Kilobytes


'44.512 Kilobytes'

See tflite.tflite model weights and size:

In [56]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_tf_lite.tflite")
interpreter.allocate_tensors()

# Get tensor details
tensor_details = interpreter.get_tensor_details()
for detail in tensor_details:
    print(f"Name: {detail['name']}, Index: {detail['index']}, Shape: {detail['shape']}, dtype: {detail['dtype']}")

# Extract weights
for detail in tensor_details:
    tensor = interpreter.tensor(detail['index'])()
    print(f"Tensor {detail['name']} - Shape: {tensor.shape}, Data: {tensor}")

Name: serving_default_quant_conv1d_6_input:0, Index: 0, Shape: [    1 48000     1], dtype: <class 'numpy.float32'>
Name: sequential_2/quant_conv1d_6/QuantConv1D/Squeeze, Index: 1, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/max_pooling1d_6/Squeeze, Index: 2, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/quant_conv1d_7/QuantConv1D/Squeeze, Index: 3, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/max_pooling1d_7/Squeeze, Index: 4, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/quant_conv1d_8/QuantConv1D/Squeeze, Index: 5, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/batch_normalization_13/batchnorm/mul, Index: 6, Shape: [ 12 664], dtype: <class 'numpy.float32'>
Name: sequential_2/batch_normalization_14/batchnorm/mul, Index: 7, Shape: [ 2 12], dtype: <class 'numpy.float32'>
Name: sequential_2/quant_conv1d_6/QuantConv1D, Index: 8, Shape: [8], dtype: <class 'numpy.float32'>
Name: sequential_2/quant_conv1d_6/QuantConv1D1,

ValueError: Tensor data is null. Run allocate_tensors() first

### Float16

In [10]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(binarized_weights_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpx1nbncsu/assets


INFO:tensorflow:Assets written to: /tmp/tmpx1nbncsu/assets
2024-09-22 15:14:44.920166: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 15:14:44.920230: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 15:14:44.920467: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpx1nbncsu
2024-09-22 15:14:44.924138: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 15:14:44.924184: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpx1nbncsu
2024-09-22 15:14:44.937093: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 15:14:45.003108: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpx1nbncsu
2024-09-22 15:14:45.031485: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/bnn/hpo/bnn_time_series_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 82.97%
Recall: 79.66%
Precision: 73.01%
F1-score: 76.19%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.7571307153369615

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.7544811567218843

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 84.35%
Recall: 84.50%
Precision: 72.47%
F1-score: 78.02%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.7820988243440998

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.7873428448662837

Time for Test dataset:
Accuracy: 84.35%
Recall: 84.50%
Precision: 72.47%
F1-score: 78.02%

Time to make a prediction for a single data point
Mean: 0.004 seconds
Max: 0.008 seconds


In [12]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 30.641 Kilobytes


'30.641 Kilobytes'

Make full int quantization of tflite.tflite model:

In [57]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(binarized_weights_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_quant = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpf_g34mu1/assets


INFO:tensorflow:Assets written to: /tmp/tmpf_g34mu1/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-27 11:31:28.172221: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-27 11:31:28.172263: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-27 11:31:28.172428: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpf_g34mu1
2024-07-27 11:31:28.176161: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-27 11:31:28.176203: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpf_g34mu1
2024-07-27 11:31:28.185280: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-27 11:31

In [58]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [59]:
# Load TFLite model and allocate tensors
# interpreter = tf.lite.Interpreter(model_content=dynamic_range_quant_model)
interpreter.allocate_tensors()

# Get tensor details
tensor_details = interpreter.get_tensor_details()
for detail in tensor_details:
    print(f"Name: {detail['name']}, Index: {detail['index']}, Shape: {detail['shape']}, dtype: {detail['dtype']}")

# Extract weights
for detail in tensor_details:
    tensor = interpreter.tensor(detail['index'])()
    print(f"Tensor {detail['name']} - Shape: {tensor.shape}, Data: {tensor}")

Name: serving_default_quant_conv1d_6_input:0, Index: 0, Shape: [    1 48000     1], dtype: <class 'numpy.uint8'>
Name: sequential_2/quant_conv1d_6/QuantConv1D/ExpandDims/dim, Index: 1, Shape: [], dtype: <class 'numpy.int32'>
Name: sequential_2/quant_conv1d_6/QuantConv1D/Squeeze, Index: 2, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/max_pooling1d_6/ExpandDims/dim, Index: 3, Shape: [], dtype: <class 'numpy.int32'>
Name: sequential_2/max_pooling1d_6/Squeeze, Index: 4, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/quant_conv1d_7/QuantConv1D/Squeeze, Index: 5, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/max_pooling1d_7/Squeeze, Index: 6, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/quant_conv1d_8/QuantConv1D/Squeeze, Index: 7, Shape: [3], dtype: <class 'numpy.int32'>
Name: sequential_2/flatten_2/Const, Index: 8, Shape: [2], dtype: <class 'numpy.int32'>
Name: sequential_2/batch_normalization_14/batchnorm/sub, Index: 9, Shape: [2], 

ValueError: Tensor data is null. Run allocate_tensors() first

In [60]:
import pathlib
import time

tflite_models_dir = pathlib.Path("../time_series_models_from_notebooks/bnn/hpo")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_bnn_full_int_file = tflite_models_dir/"bnn_time_ser_full_int_q.tflite"
tflite_bnn_full_int_file.write_bytes(tflite_model_quant)

25776

In [61]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_bnn_full_int_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_bnn_full_int_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_bnn_full_int_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_bnn_full_int_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_bnn_full_int_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_bnn_full_int_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_bnn_full_int_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_bnn_full_int_file), "KB")

Validation dataset:
Accuracy: 66.67%
Recall: 99.58%
Precision: 50.65%
F1-score: 67.14%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.6702007752656971

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.6687572000560439

Test dataset:
Accuracy: 64.54%
Recall: 98.47%
Precision: 48.08%
F1-score: 64.61%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.6443411341243627

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.6444100216284128

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.006 seconds
Min: 0.003 seconds
File size: 25.172 Kilobytes


We can see that weights from -1 +1 changed to some different numbers in full int quant model. 
But model was quantized to full int with no losses in f1-score and even better (but weights from -1 and +1 becan -127 +127)